#### install package

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import plotly.express as px
import folium
import plotly.express as px
from fbprophet import Prophet

import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
fc_df = pd.read_pickle('./시계열/fc주문_통합_수정.pkl')
display(fc_df)

,CORP_ID,REF_ORD_NO,BKG_TYP,BKG_DATE,BKG_TIME,SHPR_CD,INV_AMT,ITEM_CD,ITEM_QTY,ITEM_AMT,REF_ITEM_SEQ,DLVPREARRBRANCD,DLVPREARREMPNICKNM,DLVCLSFCD,DLVSUBCLSFCD,POST_ZONE,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2,개별가격,평균가격,metric,cluster
0,KX007,2021030365922551,7,2021-03-03,035510,90001302,22400.00000,134265392,1,14900.00000,2021030323885041,7484,C06,5Z73,1,3.0,세종특별자치시,nan,서울,서대문구,14900.00000,14900.00000,134265392,8.0
1,KX007,2021030365922951,7,2021-03-03,035521,90001441,20900.00000,4610261043,1,20900.00000,2021030323885721,6427,M09,6R67,4,50.0,경기도,광주시,경남,김해시,20900.00000,20529.68342,4610261043,0.0
2,KX007,2021030365924361,7,2021-03-03,035513,90001441,37900.00000,4936599538,1,37900.00000,2021030323887981,6158,L32,9Y25,0,53.0,경기도,광주시,경남,거제시,37900.00000,27175.94846,4936599538,8.0
3,KX007,2021030365928921,7,2021-03-03,035608,90001542,69800.00000,3858391637,1,69800.00000,2021030323895281,8909,D21,1N65,1,6.0,경기도,광주시,서울,서초구,69800.00000,102245.81445,3858391637,8.0
4,KX007,2021030365930451,7,2021-03-03,035551,90001341,73700.00000,4631307460,1,39800.00000,2021030323897651,2777,R15,3D84,1,10.0,경기도,광주시,경기,김포시,39800.00000,39800.00000,4631307460,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1744402,GP001,2021063088148031,7,2021-06-30,233214,90001776,24000.00000,4782863820,2,24000.00000,2021063049623361,9066,nan,JJ04,0,5.0,세종특별자치시,nan,서울,강동구,12000.00000,12000.00000,4782863820,3.0
1744403,GP001,2021063073024911,7,2021-06-30,104020,90001776,200000.00000,2055189125,2,200000.00000,2021063024476021,612,Q25,6N09,1j,48.0,세종특별자치시,nan,부산,해운대구,100000.00000,100000.00000,2055189125,3.0
1744404,GP001,2021063087455441,7,2021-06-30,225653,90001842,77000.00000,4891818594,2,77000.00000,2021063048425021,6155,nan,WV49,0,52.0,경기도,용인시,경남,하동군,38500.00000,38500.00000,NaN,nan
1744405,KX007,2021062966256441,7,2021-06-29,223503,90001622,125800.00000,5525868431,2,125800.00000,2021062912992741,2516,O13,6N09,2j,48.0,경기도,광주시,부산,해운대구,62900.00000,62900.00000,5525868431,6.0


In [ ]:
fc_df_shpr = fc_df.groupby(by = ['SHPR_CD', 'CORP_ID'])['REF_ORD_NO'].count()
fc_df_shpr.sort_values(ascending = False)

SHPR_CD   CORP_ID
90001302  KX007      447804
90001542  KX007      203993
90001341  KX007      168257
90001541  KX007      147525
90001441  KX007      124877
90001602  KX007      110149
90001443  KX007       99763
90001582  KX007       82117
90001622  KX007       67980
90001521  KX007       58123
90001703  GP001       56899
90001682  KX007       36227
90001702  GP001       30639
90001776  GP001       24371
90001705  GP001       24022
90001704  GP001       20352
90001768  GP001       12860
90001662  KX007        8218
90001765  GP001        6262
90001683  KX007        5855
90001381  KX007        5433
90001664  GP001        1909
90001774  GP001         730
90001842  GP001          42
Name: REF_ORD_NO, dtype: int64

In [ ]:
fc_df['year'] = fc_df['BKG_DATE'].apply(lambda x: x.year)
fc_df['month'] = fc_df['BKG_DATE'].apply(lambda x: x.month)
fc_df['day'] = fc_df['BKG_DATE'].apply(lambda x: x.day)
fc_df['weekday'] = fc_df['BKG_DATE'].apply(lambda x: x.weekday())
fc_df['week'] = fc_df['BKG_DATE'].apply(lambda x: x.isocalendar()[1])

display(fc_df)

,CORP_ID,REF_ORD_NO,BKG_TYP,BKG_DATE,BKG_TIME,SHPR_CD,INV_AMT,ITEM_CD,ITEM_QTY,ITEM_AMT,REF_ITEM_SEQ,DLVPREARRBRANCD,DLVPREARREMPNICKNM,DLVCLSFCD,DLVSUBCLSFCD,POST_ZONE,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2,개별가격,평균가격,metric,cluster,year,month,day,weekday,week
0,KX007,2021030365922551,7,2021-03-03,035510,90001302,22400.00000,134265392,1,14900.00000,2021030323885041,7484,C06,5Z73,1,3.0,세종특별자치시,nan,서울,서대문구,14900.00000,14900.00000,134265392,8.0,2021,3,3,2,9
1,KX007,2021030365922951,7,2021-03-03,035521,90001441,20900.00000,4610261043,1,20900.00000,2021030323885721,6427,M09,6R67,4,50.0,경기도,광주시,경남,김해시,20900.00000,20529.68342,4610261043,0.0,2021,3,3,2,9
2,KX007,2021030365924361,7,2021-03-03,035513,90001441,37900.00000,4936599538,1,37900.00000,2021030323887981,6158,L32,9Y25,0,53.0,경기도,광주시,경남,거제시,37900.00000,27175.94846,4936599538,8.0,2021,3,3,2,9
3,KX007,2021030365928921,7,2021-03-03,035608,90001542,69800.00000,3858391637,1,69800.00000,2021030323895281,8909,D21,1N65,1,6.0,경기도,광주시,서울,서초구,69800.00000,102245.81445,3858391637,8.0,2021,3,3,2,9
4,KX007,2021030365930451,7,2021-03-03,035551,90001341,73700.00000,4631307460,1,39800.00000,2021030323897651,2777,R15,3D84,1,10.0,경기도,광주시,경기,김포시,39800.00000,39800.00000,4631307460,8.0,2021,3,3,2,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1744402,GP001,2021063088148031,7,2021-06-30,233214,90001776,24000.00000,4782863820,2,24000.00000,2021063049623361,9066,nan,JJ04,0,5.0,세종특별자치시,nan,서울,강동구,12000.00000,12000.00000,4782863820,3.0,2021,6,30,2,26
1744403,GP001,2021063073024911,7,2021-06-30,104020,90001776,200000.00000,2055189125,2,200000.00000,2021063024476021,612,Q25,6N09,1j,48.0,세종특별자치시,nan,부산,해운대구,100000.00000,100000.00000,2055189125,3.0,2021,6,30,2,26
1744404,GP001,2021063087455441,7,2021-06-30,225653,90001842,77000.00000,4891818594,2,77000.00000,2021063048425021,6155,nan,WV49,0,52.0,경기도,용인시,경남,하동군,38500.00000,38500.00000,NaN,nan,2021,6,30,2,26
1744405,KX007,2021062966256441,7,2021-06-29,223503,90001622,125800.00000,5525868431,2,125800.00000,2021062912992741,2516,O13,6N09,2j,48.0,경기도,광주시,부산,해운대구,62900.00000,62900.00000,5525868431,6.0,2021,6,29,1,26


## 세대원수별

### 세대원수별 데이터 전처리

In [ ]:
hh_df = pd.read_csv('./외부 데이터/세대원수별 세대수 전처리.csv', encoding = 'utf-8')
display(hh_df)

,행정구역,행정구역코드,2103 전체세대,2104 전체세대,2105 전체세대,2106 전체세대,2103 1인세대 비율,2104 1인세대 비율,2105 1인세대 비율,2106 1인세대 비율,2103 2인세대 비율,2104 2인세대 비율,2105 2인세대 비율,2106 2인세대 비율,2103 3인세대 비율,2104 3인세대 비율,2105 3인세대 비율,2106 3인세대 비율,2103 4인세대 비율,2104 4인세대 비율,2105 4인세대 비율,2106 4인세대 비율,2103 5인이상세대 비율,2104 5인이상세대 비율,2105 5인이상세대 비율,2106 5인이상세대 비율
0,전국,1000000000,23157385,23157385,23225281,23256332,0.39470,0.39580,0.39660,0.39740,0.23590,0.23620,0.23670,0.23700,0.17310,0.17280,0.17250,0.17220,0.14980,0.14910,0.14840,0.14780,0.04650,0.04610,0.04580,0.04560
1,서울특별시,1100000000,4400403,4400403,4403381,4405638,0.42130,0.42230,0.42300,0.42380,0.21930,0.21960,0.22000,0.22020,0.17230,0.17210,0.17180,0.17160,0.14530,0.14460,0.14400,0.14340,0.04180,0.04140,0.04120,0.04100
2,서울특별시 종로구,1111000000,74113,74113,73712,73525,0.51100,0.51210,0.51000,0.50950,0.19550,0.19570,0.19730,0.19780,0.14240,0.14250,0.14310,0.14310,0.11150,0.11080,0.11080,0.11100,0.03960,0.03890,0.03880,0.03860
3,서울특별시 중구,1114000000,63746,63746,62847,62897,0.51010,0.51190,0.50610,0.50730,0.21480,0.21410,0.21700,0.21670,0.14140,0.14110,0.14300,0.14300,0.10020,0.09990,0.10100,0.10030,0.03350,0.03300,0.03290,0.03270
4,서울특별시 용산구,1117000000,112881,112881,112275,111650,0.48280,0.48350,0.48230,0.48100,0.21350,0.21370,0.21480,0.21590,0.15110,0.15140,0.15160,0.15200,0.11530,0.11450,0.11460,0.11440,0.03730,0.03690,0.03670,0.03670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,경상남도 거창군,4888000000,30742,30742,30679,30673,0.45910,0.46050,0.45900,0.45980,0.27070,0.27110,0.27220,0.27250,0.12810,0.12760,0.12780,0.12700,0.09750,0.09630,0.09680,0.09740,0.04460,0.04450,0.04420,0.04330
243,경상남도 합천군,4889000000,24179,24179,24256,24256,0.52200,0.52360,0.52510,0.52680,0.29130,0.29070,0.29060,0.28940,0.10000,0.09980,0.09910,0.09940,0.05660,0.05580,0.05570,0.05550,0.03010,0.03010,0.02950,0.02890
244,제주특별자치도,5000000000,302580,302580,303845,304353,0.41630,0.41710,0.41800,0.41840,0.22560,0.22560,0.22590,0.22630,0.15910,0.15890,0.15850,0.15860,0.13400,0.13380,0.13320,0.13290,0.06500,0.06460,0.06440,0.06380
245,제주특별자치도 제주시,5011000000,216717,216717,217649,218029,0.40560,0.40670,0.40770,0.40830,0.21920,0.21910,0.21940,0.21960,0.16430,0.16420,0.16370,0.16360,0.14340,0.14310,0.14250,0.14220,0.06750,0.06690,0.06670,0.06630


In [ ]:
hh_df['행정구역_시도'] = hh_df['행정구역'].apply(lambda x: x.split(' ')[0])
hh_df['행정구역_시군구'] = hh_df['행정구역'].apply(lambda x: x.split(' ')[1])

In [ ]:
hh_df.head()

,행정구역,행정구역코드,2103 전체세대,2104 전체세대,2105 전체세대,2106 전체세대,2103 1인세대 비율,2104 1인세대 비율,2105 1인세대 비율,2106 1인세대 비율,2103 2인세대 비율,2104 2인세대 비율,2105 2인세대 비율,2106 2인세대 비율,2103 3인세대 비율,2104 3인세대 비율,2105 3인세대 비율,2106 3인세대 비율,2103 4인세대 비율,2104 4인세대 비율,2105 4인세대 비율,2106 4인세대 비율,2103 5인이상세대 비율,2104 5인이상세대 비율,2105 5인이상세대 비율,2106 5인이상세대 비율,행정구역_시도,행정구역_시군구
0,전국,1000000000,23157385,23157385,23225281,23256332,0.39470,0.39580,0.39660,0.39740,0.23590,0.23620,0.23670,0.23700,0.17310,0.17280,0.17250,0.17220,0.14980,0.14910,0.14840,0.14780,0.04650,0.04610,0.04580,0.04560,전국,
1,서울특별시,1100000000,4400403,4400403,4403381,4405638,0.42130,0.42230,0.42300,0.42380,0.21930,0.21960,0.22000,0.22020,0.17230,0.17210,0.17180,0.17160,0.14530,0.14460,0.14400,0.14340,0.04180,0.04140,0.04120,0.04100,서울특별시,
2,서울특별시 종로구,1111000000,74113,74113,73712,73525,0.51100,0.51210,0.51000,0.50950,0.19550,0.19570,0.19730,0.19780,0.14240,0.14250,0.14310,0.14310,0.11150,0.11080,0.11080,0.11100,0.03960,0.03890,0.03880,0.03860,서울특별시,종로구
3,서울특별시 중구,1114000000,63746,63746,62847,62897,0.51010,0.51190,0.50610,0.50730,0.21480,0.21410,0.21700,0.21670,0.14140,0.14110,0.14300,0.14300,0.10020,0.09990,0.10100,0.10030,0.03350,0.03300,0.03290,0.03270,서울특별시,중구
4,서울특별시 용산구,1117000000,112881,112881,112275,111650,0.48280,0.48350,0.48230,0.48100,0.21350,0.21370,0.21480,0.21590,0.15110,0.15140,0.15160,0.15200,0.11530,0.11450,0.11460,0.11440,0.03730,0.03690,0.03670,0.03670,서울특별시,용산구


In [ ]:
df2 = hh_df[hh_df['행정구역_시군구'] == '']
df2

,행정구역,행정구역코드,2103 전체세대,2104 전체세대,2105 전체세대,2106 전체세대,2103 1인세대 비율,2104 1인세대 비율,2105 1인세대 비율,2106 1인세대 비율,2103 2인세대 비율,2104 2인세대 비율,2105 2인세대 비율,2106 2인세대 비율,2103 3인세대 비율,2104 3인세대 비율,2105 3인세대 비율,2106 3인세대 비율,2103 4인세대 비율,2104 4인세대 비율,2105 4인세대 비율,2106 4인세대 비율,2103 5인이상세대 비율,2104 5인이상세대 비율,2105 5인이상세대 비율,2106 5인이상세대 비율,행정구역_시도,행정구역_시군구
0,전국,1000000000,23157385,23157385,23225281,23256332,0.39470,0.39580,0.39660,0.39740,0.23590,0.23620,0.23670,0.23700,0.17310,0.17280,0.17250,0.17220,0.14980,0.14910,0.14840,0.14780,0.04650,0.04610,0.04580,0.04560,전국,
1,서울특별시,1100000000,4400403,4400403,4403381,4405638,0.42130,0.42230,0.42300,0.42380,0.21930,0.21960,0.22000,0.22020,0.17230,0.17210,0.17180,0.17160,0.14530,0.14460,0.14400,0.14340,0.04180,0.04140,0.04120,0.04100,서울특별시,
27,부산광역시,2600000000,1527830,1527830,1532053,1533514,0.38780,0.38900,0.39040,0.39150,0.25210,0.25250,0.25280,0.25320,0.17890,0.17850,0.17790,0.17760,0.14050,0.13980,0.13890,0.13810,0.04070,0.04020,0.04000,0.03960,부산광역시,
44,대구광역시,2700000000,1060866,1060866,1061872,1059826,0.37160,0.37280,0.37350,0.37310,0.24300,0.24350,0.24390,0.24470,0.18500,0.18450,0.18430,0.18430,0.15740,0.15660,0.15590,0.15570,0.04300,0.04260,0.04240,0.04220,대구광역시,
53,인천광역시,2800000000,1272361,1272361,1277842,1280291,0.36610,0.36750,0.36880,0.36980,0.23350,0.23400,0.23450,0.23490,0.18870,0.18830,0.18770,0.18740,0.16370,0.16260,0.16180,0.16090,0.04800,0.04760,0.04720,0.04700,인천광역시,
64,광주광역시,2900000000,636160,636160,637364,638270,0.38700,0.38830,0.38880,0.38980,0.22990,0.23000,0.23070,0.23100,0.17350,0.17320,0.17290,0.17260,0.15720,0.15670,0.15620,0.15560,0.05240,0.05180,0.05140,0.05100,광주광역시,
70,대전광역시,3000000000,655196,655196,657087,657781,0.40100,0.40190,0.40320,0.40430,0.23080,0.23140,0.23150,0.23180,0.17080,0.17050,0.17020,0.16980,0.15050,0.14960,0.14870,0.14830,0.04690,0.04660,0.04640,0.04580,대전광역시,
76,울산광역시,3100000000,478528,478528,479362,479971,0.34130,0.34230,0.34350,0.34460,0.23800,0.23860,0.23910,0.23970,0.19690,0.19640,0.19580,0.19530,0.17880,0.17800,0.17720,0.17620,0.04500,0.04470,0.04440,0.04420,울산광역시,
82,세종특별자치시,3600000000,146763,146763,147848,148604,0.33670,0.33780,0.33890,0.34110,0.21060,0.21020,0.21080,0.21060,0.18990,0.18980,0.18940,0.18860,0.20070,0.20030,0.19930,0.19870,0.06210,0.06190,0.06160,0.06100,세종특별자치시,
83,세종특별자치시,3611000000,146763,146763,147848,148604,0.33670,0.33780,0.33890,0.34110,0.21060,0.21020,0.21080,0.21060,0.18990,0.18980,0.18940,0.18860,0.20070,0.20030,0.19930,0.19870,0.06210,0.06190,0.06160,0.06100,세종특별자치시,


In [ ]:
df2 = df2.drop(index=82, axis=0)

In [ ]:
## 행정구역_시도 이름 변경
df2.행정구역_시도.replace({'서울특별시':'서울', '부산광역시':'부산', '대전광역시':'대전', 
                     '인천광역시':'인천','대구광역시':'대구','광주광역시':'광주',
                     '제주특별자치도':'제주','울산광역시':'울산','세종특별자치시':'세종', 
                     '경상남도': '경남', '경기도':'경기','전라북도':'전북','충청남도':'충남', 
                     '경상북도':'경북','강원도':'강원', '전라남도':'전남','충청북도':'충북',
                     '대구시':'대구','부산시':'부산','인천시':'인천'},inplace=True)

In [ ]:
df2 = df2[df2['행정구역_시도']!='전국']
df2.head()

,행정구역,행정구역코드,2103 전체세대,2104 전체세대,2105 전체세대,2106 전체세대,2103 1인세대 비율,2104 1인세대 비율,2105 1인세대 비율,2106 1인세대 비율,2103 2인세대 비율,2104 2인세대 비율,2105 2인세대 비율,2106 2인세대 비율,2103 3인세대 비율,2104 3인세대 비율,2105 3인세대 비율,2106 3인세대 비율,2103 4인세대 비율,2104 4인세대 비율,2105 4인세대 비율,2106 4인세대 비율,2103 5인이상세대 비율,2104 5인이상세대 비율,2105 5인이상세대 비율,2106 5인이상세대 비율,행정구역_시도,행정구역_시군구
1,서울특별시,1100000000,4400403,4400403,4403381,4405638,0.42130,0.42230,0.42300,0.42380,0.21930,0.21960,0.22000,0.22020,0.17230,0.17210,0.17180,0.17160,0.14530,0.14460,0.14400,0.14340,0.04180,0.04140,0.04120,0.04100,서울,
27,부산광역시,2600000000,1527830,1527830,1532053,1533514,0.38780,0.38900,0.39040,0.39150,0.25210,0.25250,0.25280,0.25320,0.17890,0.17850,0.17790,0.17760,0.14050,0.13980,0.13890,0.13810,0.04070,0.04020,0.04000,0.03960,부산,
44,대구광역시,2700000000,1060866,1060866,1061872,1059826,0.37160,0.37280,0.37350,0.37310,0.24300,0.24350,0.24390,0.24470,0.18500,0.18450,0.18430,0.18430,0.15740,0.15660,0.15590,0.15570,0.04300,0.04260,0.04240,0.04220,대구,
53,인천광역시,2800000000,1272361,1272361,1277842,1280291,0.36610,0.36750,0.36880,0.36980,0.23350,0.23400,0.23450,0.23490,0.18870,0.18830,0.18770,0.18740,0.16370,0.16260,0.16180,0.16090,0.04800,0.04760,0.04720,0.04700,인천,
64,광주광역시,2900000000,636160,636160,637364,638270,0.38700,0.38830,0.38880,0.38980,0.22990,0.23000,0.23070,0.23100,0.17350,0.17320,0.17290,0.17260,0.15720,0.15670,0.15620,0.15560,0.05240,0.05180,0.05140,0.05100,광주,


In [ ]:
df2 = df2[['행정구역_시도','2103 전체세대', '2104 전체세대', '2105 전체세대', '2106 전체세대',
       '2103 1인세대 비율', '2104 1인세대 비율', '2105 1인세대 비율', '2106 1인세대 비율',
       '2103 2인세대 비율', '2104 2인세대 비율', '2105 2인세대 비율', '2106 2인세대 비율',
       '2103 3인세대 비율', '2104 3인세대 비율', '2105 3인세대 비율', '2106 3인세대 비율',
       '2103 4인세대 비율', '2104 4인세대 비율', '2105 4인세대 비율', '2106 4인세대 비율',
       '2103 5인이상세대 비율', '2104 5인이상세대 비율', '2105 5인이상세대 비율', '2106 5인이상세대 비율']]

In [ ]:
melt_df = pd.melt(df2, id_vars=['행정구역_시도'], var_name= '월별세대수', value_name='세대수')
melt_df.head()

,행정구역_시도,월별세대수,세대수
0,서울,2103 전체세대,4400403.00000
1,부산,2103 전체세대,1527830.00000
2,대구,2103 전체세대,1060866.00000
3,인천,2103 전체세대,1272361.00000
4,광주,2103 전체세대,636160.00000


In [ ]:
melt_df['month'] = melt_df['월별세대수'].apply(lambda x : x[2:4])
melt_df['세대원수'] = melt_df['월별세대수'].apply(lambda x : x[5:7])

In [ ]:
melt_df.head()

,행정구역_시도,월별세대수,세대수,month,세대원수
0,서울,2103 전체세대,4400403.00000,03,전체
1,부산,2103 전체세대,1527830.00000,03,전체
2,대구,2103 전체세대,1060866.00000,03,전체
3,인천,2103 전체세대,1272361.00000,03,전체
4,광주,2103 전체세대,636160.00000,03,전체


In [ ]:
melt_df = melt_df[['행정구역_시도', '세대수', 'month', '세대원수']]

In [ ]:
df = melt_df.pivot(index = ['행정구역_시도','month'], columns = '세대원수', values='세대수')

### 세대원수별 최종 데이터

In [ ]:
df = df.reset_index()

In [ ]:
df.head()

세대원수,행정구역_시도,month,1인,2인,3인,4인,5인,전체
0,강원,03,0.43360,0.25750,0.15220,0.11540,0.04130,735644.00000
1,강원,04,0.43440,0.25780,0.15190,0.11470,0.04120,735644.00000
2,강원,05,0.43440,0.25850,0.15180,0.11440,0.04090,736980.00000
3,강원,06,0.43500,0.25900,0.15150,0.11390,0.04060,738068.00000
4,경기,03,0.35980,0.22390,0.18700,0.17670,0.05260,5719333.00000


In [ ]:
df.columns = ['시도', 'month', '1인', '2인', '3인', '4인', '5인', '전체']

In [ ]:
df.to_pickle('지역별세대원수별인구수.pkl')

## 성/연령별

### 성/연령별 데이터 전처리

In [ ]:
ys_df = pd.read_csv('./외부 데이터/연령별_성별_인구_전처리_1026.csv', encoding = 'utf-8')
display(ys_df.head())

,행정구역,행정구역코드,2103 총인구수,2104 총인구수,2105 총인구수,2106 총인구수,2103 10대 미만 비율,2104 10대 미만 비율,2105 10대 미만 비율,2106 10대 미만 비율,2103 10대 비율,2104 10대 비율,2105 10대 비율,2106 10대 비율,2103 20대 비율,2104 20대 비율,2105 20대 비율,2106 20대 비율,2103 30대 비율,2104 30대 비율,2105 30대 비율,2106 30대 비율,2103 40대 비율,2104 40대 비율,2105 40대 비율,2106 40대 비율,2103 50대 비율,2104 50대 비율,2105 50대 비율,2106 50대 비율,2103 60대 비율,2104 60대 비율,2105 60대 비율,2106 60대 비율,2103 70대 비율,2104 70대 비율,2105 70대 비율,2106 70대 비율,2103 80대 비율,2104 80대 비율,2105 80대 비율,2106 80대 비율,2103 90대 비율,2104 90대 비율,2105 90대 비율,2106 90대 비율,2103 90대 초과 비율,2104 90대 초과 비율,2105 90대 초과 비율,2106 90대 초과 비율,2103 남성 비율,2104 남성 비율,2105 남성 비율,2106 남성 비율,2103 여성 비율,2104 여성 비율,2105 여성 비율,2106 여성 비율
0,서울특별시,1100000000,9598484,9588711,9575355,9565990,0.06370,0.06340,0.06320,0.06300,0.08060,0.08040,0.08020,0.08010,0.15110,0.15100,0.15100,0.15090,0.15220,0.15210,0.15200,0.15180,0.15750,0.15730,0.15700,0.15690,0.15770,0.15760,0.15770,0.15750,0.13080,0.13140,0.13200,0.13270,0.07270,0.07280,0.07280,0.07280,0.02920,0.02940,0.02960,0.02980,0.00420,0.00430,0.00430,0.00430,0.00030,0.00030,0.00030,0.00030,0.48630,0.48620,0.48610,0.48600,0.51370,0.51380,0.51390,0.51400
1,서울특별시 종로구,1111000000,147296,147113,146377,146029,0.04970,0.04970,0.04970,0.04980,0.07430,0.07380,0.07370,0.07320,0.15800,0.15800,0.15860,0.15850,0.13400,0.13410,0.13420,0.13420,0.14780,0.14740,0.14690,0.14680,0.17090,0.17080,0.17100,0.17080,0.13610,0.13700,0.13750,0.13830,0.08200,0.08160,0.08080,0.08070,0.04040,0.04070,0.04070,0.04100,0.00640,0.00650,0.00650,0.00640,0.00040,0.00040,0.00040,0.00040,0.48640,0.48650,0.48640,0.48610,0.51360,0.51350,0.51360,0.51390
2,서울특별시 중구,1114000000,124552,124477,123133,123016,0.05310,0.05290,0.05310,0.05290,0.05400,0.05380,0.05410,0.05420,0.14810,0.14760,0.14910,0.14890,0.15960,0.16020,0.16190,0.16160,0.14330,0.14330,0.14390,0.14420,0.16460,0.16430,0.16480,0.16420,0.14320,0.14360,0.14360,0.14430,0.08580,0.08550,0.08390,0.08390,0.03960,0.03990,0.03860,0.03880,0.00720,0.00730,0.00650,0.00650,0.00150,0.00150,0.00050,0.00050,0.48860,0.48830,0.48810,0.48790,0.51140,0.51170,0.51190,0.51210
3,서울특별시 용산구,1117000000,229013,228755,227515,226378,0.05830,0.05800,0.05800,0.05810,0.06710,0.06680,0.06680,0.06690,0.14750,0.14750,0.14800,0.14840,0.16750,0.16760,0.16810,0.16860,0.15420,0.15410,0.15390,0.15360,0.16090,0.16070,0.16090,0.16090,0.12660,0.12710,0.12720,0.12750,0.07350,0.07350,0.07290,0.07210,0.03750,0.03770,0.03750,0.03740,0.00620,0.00630,0.00620,0.00610,0.00050,0.00050,0.00050,0.00040,0.48110,0.48100,0.48130,0.48130,0.51890,0.51900,0.51870,0.51870
4,서울특별시 성동구,1120000000,291906,291534,289656,289162,0.06840,0.06840,0.06870,0.06860,0.06740,0.06710,0.06720,0.06710,0.15300,0.15300,0.15360,0.15350,0.16530,0.16510,0.16510,0.16500,0.15270,0.15240,0.15210,0.15210,0.16140,0.16130,0.16130,0.16090,0.12710,0.12770,0.12780,0.12840,0.07070,0.07070,0.07000,0.07000,0.02970,0.03000,0.03010,0.03030,0.00390,0.00390,0.00390,0.00390,0.00030,0.00030,0.00030,0.00030,0.48820,0.48800,0.48800,0.48780,0.51180,0.51200,0.51200,0.51220


In [ ]:
ys_df['행정구역_시도'] = ys_df['행정구역'].apply(lambda x: x.split(' ')[0])
ys_df['행정구역_시군구'] = ys_df['행정구역'].apply(lambda x: x.split(' ')[1])

In [ ]:
df5 = ys_df[ys_df['행정구역_시군구'] == '']

In [ ]:
df5.행정구역_시도.replace({'서울특별시':'서울', '부산광역시':'부산', '대전광역시':'대전', 
                     '인천광역시':'인천','대구광역시':'대구','광주광역시':'광주',
                     '제주특별자치도':'제주','울산광역시':'울산','세종특별자치시':'세종', 
                     '경상남도': '경남', '경기도':'경기','전라북도':'전북','충청남도':'충남', 
                     '경상북도':'경북','강원도':'강원', '전라남도':'전남','충청북도':'충북',
                     '대구시':'대구','부산시':'부산','인천시':'인천'},inplace=True)

In [ ]:
df5

,행정구역,행정구역코드,2103 총인구수,2104 총인구수,2105 총인구수,2106 총인구수,2103 10대 미만 비율,2104 10대 미만 비율,2105 10대 미만 비율,2106 10대 미만 비율,2103 10대 비율,2104 10대 비율,2105 10대 비율,2106 10대 비율,2103 20대 비율,2104 20대 비율,2105 20대 비율,2106 20대 비율,2103 30대 비율,2104 30대 비율,2105 30대 비율,2106 30대 비율,2103 40대 비율,2104 40대 비율,2105 40대 비율,2106 40대 비율,2103 50대 비율,2104 50대 비율,2105 50대 비율,2106 50대 비율,2103 60대 비율,2104 60대 비율,2105 60대 비율,2106 60대 비율,2103 70대 비율,2104 70대 비율,2105 70대 비율,2106 70대 비율,2103 80대 비율,2104 80대 비율,2105 80대 비율,2106 80대 비율,2103 90대 비율,2104 90대 비율,2105 90대 비율,2106 90대 비율,2103 90대 초과 비율,2104 90대 초과 비율,2105 90대 초과 비율,2106 90대 초과 비율,2103 남성 비율,2104 남성 비율,2105 남성 비율,2106 남성 비율,2103 여성 비율,2104 여성 비율,2105 여성 비율,2106 여성 비율,행정구역_시도,행정구역_시군구
0,서울특별시,1100000000,9598484,9588711,9575355,9565990,0.06370,0.06340,0.06320,0.06300,0.08060,0.08040,0.08020,0.08010,0.15110,0.15100,0.15100,0.15090,0.15220,0.15210,0.15200,0.15180,0.15750,0.15730,0.15700,0.15690,0.15770,0.15760,0.15770,0.15750,0.13080,0.13140,0.13200,0.13270,0.07270,0.07280,0.07280,0.07280,0.02920,0.02940,0.02960,0.02980,0.00420,0.00430,0.00430,0.00430,0.00030,0.00030,0.00030,0.00030,0.48630,0.48620,0.48610,0.48600,0.51370,0.51380,0.51390,0.51400,서울,
26,부산광역시,2600000000,3372399,3369704,3367334,3364358,0.06780,0.06760,0.06730,0.06710,0.08000,0.07990,0.07970,0.07970,0.12660,0.12650,0.12630,0.12600,0.12270,0.12260,0.12240,0.12220,0.15060,0.15060,0.15030,0.15040,0.16670,0.16630,0.16610,0.16570,0.15920,0.15980,0.16060,0.16130,0.08690,0.08710,0.08730,0.08740,0.03490,0.03520,0.03540,0.03560,0.00430,0.00440,0.00440,0.00440,0.00020,0.00020,0.00020,0.00020,0.48980,0.48970,0.48970,0.48960,0.51020,0.51030,0.51030,0.51040,부산,
43,대구광역시,2700000000,2408875,2406296,2402940,2397646,0.07270,0.07230,0.07210,0.07200,0.09260,0.09240,0.09230,0.09220,0.13220,0.13200,0.13190,0.13180,0.12080,0.12060,0.12040,0.12020,0.15790,0.15780,0.15730,0.15730,0.17560,0.17540,0.17550,0.17520,0.13790,0.13870,0.13950,0.14030,0.07310,0.07310,0.07330,0.07330,0.03300,0.03320,0.03340,0.03350,0.00410,0.00410,0.00410,0.00400,0.00020,0.00020,0.00020,0.00020,0.49320,0.49310,0.49310,0.49310,0.50680,0.50690,0.50690,0.50690,대구,
52,인천광역시,2800000000,2936461,2936214,2936382,2936367,0.07660,0.07620,0.07590,0.07560,0.09290,0.09280,0.09260,0.09250,0.13550,0.13530,0.13500,0.13470,0.13710,0.13690,0.13670,0.13650,0.16400,0.16400,0.16360,0.16370,0.17370,0.17350,0.17350,0.17320,0.12850,0.12950,0.13050,0.13150,0.06050,0.06050,0.06060,0.06060,0.02680,0.02700,0.02710,0.02730,0.00420,0.00420,0.00430,0.00430,0.00020,0.00020,0.00020,0.00020,0.50090,0.50080,0.50080,0.50080,0.49910,0.49920,0.49920,0.49920,인천,
63,광주광역시,2900000000,1445473,1444787,1443154,1442647,0.08150,0.08120,0.08090,0.08070,0.10740,0.10720,0.10700,0.10680,0.14410,0.14410,0.14410,0.14410,0.12740,0.12720,0.12690,0.12670,0.16580,0.16570,0.16500,0.16490,0.16180,0.16180,0.16230,0.16210,0.11480,0.11540,0.11600,0.11670,0.06480,0.06480,0.06490,0.06510,0.02800,0.02820,0.02840,0.02860,0.00410,0.00420,0.00410,0.00410,0.00020,0.00020,0.00020,0.00020,0.49480,0.49480,0.49480,0.49480,0.50520,0.50520,0.50520,0.50520,광주,
69,대전광역시,3000000000,1458463,1457619,1457161,1456107,0.07700,0.07670,0.07630,0.07600,0.09940,0.09920,0.09900,0.09880,0.14500,0.14500,0.14500,0.14490,0.13190,0.13180,0.13160,0.13130,0.16120,0.16100,0.16050,0.16050,0.16470,0.16450,0.16460,0.16430,0.12570,0.12650,0.12740,0.12830,0.06270,0.06270,0.06280,0.06290,0.02820,0.02840,0.02860,0.02880,0.00410,0.00410,0.00410,0.00410,0.00010,0.00010,0.00010,0.00010,0.49920,0.49910,0.49910,0.49900,0.50080,0.50090,0.50090,0.50100,대전,
75,울산광역시,3100000000,1129254,1128163,1127175,1126369,0.08540,0.08500,0.08460,0.08420,0.09830,0.09830,0.09820,0.09820,0.12610,0.12590,0.12560,0.12530,0.13230,0.13200,0.13170,0.13130,0.16500,0.16500,0.16460,0.16450,0.18400,0.18380,0.18380,0.18360,0.13170,0.13260,0.13360,0.13460,0.05380,0.05410,0.05430,0.05450,0.02040,0.02050,0.02070,0.02080,0.00270,0.00270,0.00280,0.00280,0.00010,0.00010,0.00010,0.00010,0.51400,0.51400,0.51400,0.51400,0.48600,0.48600,0.48600

In [ ]:
df5.drop(index = 82, axis=1, inplace=True)

In [ ]:
for i in ['2103','2104','2105','2106']:
  df5['%s 2,30대 비율'%i] = df5['%s 20대 비율'%i]+df5['%s 30대 비율'%i]
  df5['%s 4,5,60대 비율'%i] = df5['%s 40대 비율'%i] + df5['%s 50대 비율'%i] + df5['%s 60대 비율'%i]

In [ ]:
df5 = df5[['2103 총인구수', '2104 총인구수', '2105 총인구수', '2106 총인구수',
       '2103 10대 비율', '2104 10대 비율', '2105 10대 비율', '2106 10대 비율',
       '2103 2,30대 비율', '2104 2,30대 비율', '2105 2,30대 비율', '2106 2,30대 비율',
       '2103 4,5,60대 비율', '2104 4,5,60대 비율', '2105 4,5,60대 비율',
       '2106 4,5,60대 비율', '2103 남성 비율', '2104 남성 비율', '2105 남성 비율',
       '2106 남성 비율', '2103 여성 비율', '2104 여성 비율', '2105 여성 비율', '2106 여성 비율',
       '행정구역_시도']]

In [ ]:
melt_df1 = pd.melt(df5, id_vars=['행정구역_시도'], var_name= '성연령', value_name='인구수')

In [ ]:
melt_df1

,행정구역_시도,성연령,인구수
0,서울,2103 총인구수,9598484.00000
1,부산,2103 총인구수,3372399.00000
2,대구,2103 총인구수,2408875.00000
3,인천,2103 총인구수,2936461.00000
4,광주,2103 총인구수,1445473.00000
...,...,...,...
403,전북,2106 여성 비율,0.50250
404,전남,2106 여성 비율,0.49680
405,경북,2106 여성 비율,0.49600
406,경남,2106 여성 비율,0.49650


In [ ]:
melt_df1['month'] = melt_df1['성연령'].apply(lambda x : x[2:4])

In [ ]:
melt_df1['성연령분류'] = melt_df1['성연령'].apply(lambda x : x[5:])

In [ ]:
melt_df1

,행정구역_시도,성연령,인구수,month,성연령분류
0,서울,2103 총인구수,9598484.00000,03,총인구수
1,부산,2103 총인구수,3372399.00000,03,총인구수
2,대구,2103 총인구수,2408875.00000,03,총인구수
3,인천,2103 총인구수,2936461.00000,03,총인구수
4,광주,2103 총인구수,1445473.00000,03,총인구수
...,...,...,...,...,...
403,전북,2106 여성 비율,0.50250,06,여성 비율
404,전남,2106 여성 비율,0.49680,06,여성 비율
405,경북,2106 여성 비율,0.49600,06,여성 비율
406,경남,2106 여성 비율,0.49650,06,여성 비율


In [ ]:
melt_df1 = melt_df1[['행정구역_시도', '인구수', 'month', '성연령분류']]

In [ ]:
df_y = melt_df1.pivot(index = ['행정구역_시도','month'], columns = '성연령분류', values='인구수')

In [ ]:
df_y

성연령분류          10대 비율  2,30대 비율  4,5,60대 비율   남성 비율   여성 비율           총인구수
행정구역_시도 month                                                             
강원      03    0.08860   0.22280     0.47770 0.50330 0.49670  1536399.00000
        04    0.08840   0.22230     0.47850 0.50320 0.49680  1536175.00000
        05    0.08820   0.22200     0.47920 0.50340 0.49660  1535491.00000
        06    0.08810   0.22170     0.47990 0.50340 0.49660  1535530.00000
경기      03    0.09840   0.27590     0.45420 0.50330 0.49670 13465837.00000
...               ...       ...         ...     ...     ...            ...
충남      06    0.09500   0.23780     0.45180 0.51110 0.48890  2117400.00000
충북      03    0.09270   0.24440     0.46380 0.50730 0.49270  1595714.00000
        04    0.09260   0.24430     0.46430 0.50720 0.49280  1596303.00000
        05    0.09240   0.24400     0.46500 0.50720 0.49280  1596955.00000
        06    0.09220   0.24380     0.46550 0.50720 0.49280  1597503.00000

[68 rows x 6 columns]

### 성/연령별 최종 데이터

In [ ]:
df_y = df_y.reset_index()

In [ ]:
df_y.columns

Index(['행정구역_시도', 'month', '10대 비율', '2,30대 비율', '4,5,60대 비율', '남성 비율',
       '여성 비율', '총인구수'],
      dtype='object', name='성연령분류')

In [ ]:
df_y.columns = ['시도', 'month', '10대 비율', '2,30대 비율', '4,5,60대 비율', '남성 비율',
       '여성 비율', '총인구수']

In [ ]:
df_y.to_pickle('지역별성연령별인구수.pkl')

## 최종 외부 데이터

In [ ]:
final = pd.merge(df_y, df, how = 'inner', on=['시도','month'] )

In [ ]:
final.head()

,시도,month,10대 비율,"2,30대 비율","4,5,60대 비율",남성 비율,여성 비율,총인구수,1인,2인,3인,4인,5인,전체
0,강원,03,0.08860,0.22280,0.47770,0.50330,0.49670,1536399.00000,0.43360,0.25750,0.15220,0.11540,0.04130,735644.00000
1,강원,04,0.08840,0.22230,0.47850,0.50320,0.49680,1536175.00000,0.43440,0.25780,0.15190,0.11470,0.04120,735644.00000
2,강원,05,0.08820,0.22200,0.47920,0.50340,0.49660,1535491.00000,0.43440,0.25850,0.15180,0.11440,0.04090,736980.00000
3,강원,06,0.08810,0.22170,0.47990,0.50340,0.49660,1535530.00000,0.43500,0.25900,0.15150,0.11390,0.04060,738068.00000
4,경기,03,0.09840,0.27590,0.45420,0.50330,0.49670,13465837.00000,0.35980,0.22390,0.18700,0.17670,0.05260,5719333.00000


In [ ]:
final.columns

Index(['시도', 'month', '10대 비율', '2,30대 비율', '4,5,60대 비율', '남성 비율', '여성 비율',
       '총인구수', '1인', '2인', '3인', '4인', '5인', '전체'],
      dtype='object')

In [ ]:
final.columns = ['시도', 'month', '10대 비율', '2,30대 비율', '4,5,60대 비율', '남성 비율', '여성 비율',
       '총인구수', '1인', '2인', '3인', '4인', '5인', '전체가구수']

In [ ]:
final.to_csv("demographic.csv", encoding='utf-8')

In [ ]:
final.to_pickle("demographic.pkl")